# Content
1. Import Libraries
2. Import Data
3. Data Prep for Customer data 
     - data wrangling 
     - data consistency 
     - combining 

# Importing Libraries

In [2]:
#Import libraries
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

# Importing Data

In [3]:
path = r'C:\Users\tlayt\OneDrive\Documents\CF Data Analytics\Acheivement 4\Instacart Basket Analysis'

In [4]:
#import data
cust = pd.read_csv(os.path.join(path, '02 Data', 'Original Data', 'customers.csv'), index_col = False)


In [5]:
op_agged = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_agged.pkl'))


# Data Prep

## Wrangling

In [6]:
cust.shape

(206209, 10)

In [7]:
cust.head()

,user_id,First Name,Surnam,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


In [8]:
cust.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206209 entries, 0 to 206208
Data columns (total 10 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   user_id       206209 non-null  int64 
 1   First Name    194950 non-null  object
 2   Surnam        206209 non-null  object
 3   Gender        206209 non-null  object
 4   STATE         206209 non-null  object
 5   Age           206209 non-null  int64 
 6   date_joined   206209 non-null  object
 7   n_dependants  206209 non-null  int64 
 8   fam_status    206209 non-null  object
 9   income        206209 non-null  int64 
dtypes: int64(4), object(6)
memory usage: 15.7+ MB


In [9]:
cust.describe()

,user_id,Age,n_dependants,income
count,206209.000000,206209.000000,206209.000000,206209.000000
mean,103105.000000,49.501646,1.499823,94632.852548
std,59527.555167,18.480962,1.118433,42473.786988
min,1.000000,18.000000,0.000000,25903.000000
25%,51553.000000,33.000000,0.000000,59874.000000
50%,103105.000000,49.000000,1.000000,93547.000000
75%,154657.000000,66.000000,3.000000,124244.000000
max,206209.000000,81.000000,3.000000,593901.000000


In [10]:
#renaming user id to customer id to match other df
cust.rename(columns = {'user_id' : 'customer_id'}, inplace = True)

In [11]:
#renaming surname to last name for consistency 
cust.rename(columns = {'Surnam' : 'Last_Name'}, inplace = True)

In [15]:
#renaming first name to first_name for cunsistency  
cust.rename(columns = {'First Name' : 'First_Name'}, inplace = True)

In [13]:
#renaming STATE to State 
cust.rename(columns = {'STATE' : 'State'}, inplace = True)

In [16]:
#checking
cust.head()

,customer_id,First_Name,Last_Name,Gender,State,Age,date_joined,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


In [17]:
#changing customer id datatype to string instead of integer 
cust['customer_id'] = cust['customer_id'].astype('str')

In [18]:
cust['customer_id'].dtype

dtype('O')

## Quality and Consistency Checks 

In [19]:
#checking for mixed data types 
for col in cust.columns.tolist():
  weird = (cust[[col]].applymap(type) != cust[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (cust[weird]) > 0:
    print (col)

First_Name


In [21]:
#Changing first name to string data type
cust['First_Name'] = cust['First_Name'].astype('str')

In [22]:
#checking
cust['First_Name'].dtype

dtype('O')

In [23]:
#checking for missing values
cust.isnull().sum()

customer_id     0
First_Name      0
Last_Name       0
Gender          0
State           0
Age             0
date_joined     0
n_dependants    0
fam_status      0
income          0
dtype: int64

In [24]:
#checking for dups 
cust_dups = cust[cust.duplicated()]

In [25]:
cust_dups

,customer_id,First_Name,Last_Name,Gender,State,Age,date_joined,n_dependants,fam_status,income


In [26]:
cust.head()

,customer_id,First_Name,Last_Name,Gender,State,Age,date_joined,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


In [27]:
#checking max income values 
cust.loc[cust['income'] >= 300000]

,customer_id,First_Name,Last_Name,Gender,State,Age,date_joined,n_dependants,fam_status,income
434,159362,Tina,Shannon,Female,Missouri,74,1/3/2017,3,married,372334
979,200930,Charles,Nichols,Male,South Carolina,60,1/6/2017,1,married,300913
991,136298,Kevin,Ortega,Male,New Mexico,47,1/6/2017,3,married,433206
1903,89592,Katherine,Abbott,Female,Minnesota,22,1/12/2017,0,single,317122
2018,33595,Jose,Castaneda,Male,Mississippi,65,1/12/2017,2,married,577728
...,...,...,...,...,...,...,...,...,...,...
202655,98580,nan,Patterson,Male,Washington,43,3/12/2020,2,married,592409
202741,100969,Douglas,Campos,Male,Rhode Island,77,3/13/2020,0,divorced/widowed,443380
203226,183344,Louise,Leal,Female,Wisconsin,29,3/15/2020,3,married,374539
203425,144788,Henry,Kemp,Male,Wisconsin,64,3/16/2020,1,married,400526


In [28]:
#creating subset of data without 
cust_sub = cust.loc[cust['income'] < 300000]

In [29]:
#checking
cust_sub.shape

(205827, 10)

In [31]:
#exporting cust_sub
cust_sub.to_csv(os.path.join(path, '02 Data','Prepared Data', 'Customer_no_outliers.csv'))

In [32]:
#exporting clean cust 
cust.to_csv(os.path.join(path, '02 Data','Prepared Data', 'Customer_Clean.csv'))

# Combining Dataframes 

In [46]:
op_agged.columns

Index(['order_id', 'customer_id', 'order_number', 'orders_day_of_week',
       'order_hour_of_day', 'days_since_prior_order', 'product_id',
       'add_to_cart_order', 'reordered', 'product_name', 'aisle_id',
       'department_id', 'prices', '_merge', 'price_labels', 'busiest_day',
       'busiest_days', 'busiest_period_of_day', 'max_order', 'loyalty_flag',
       'avg_spending', 'spending_flag', 'med_DSPO', 'frequency_flag'],
      dtype='object')

In [47]:
#dropping _merge column
op_agged = op_agged.drop(columns = ['_merge'])

In [48]:
#checking
op_agged.columns

Index(['order_id', 'customer_id', 'order_number', 'orders_day_of_week',
       'order_hour_of_day', 'days_since_prior_order', 'product_id',
       'add_to_cart_order', 'reordered', 'product_name', 'aisle_id',
       'department_id', 'prices', 'price_labels', 'busiest_day',
       'busiest_days', 'busiest_period_of_day', 'max_order', 'loyalty_flag',
       'avg_spending', 'spending_flag', 'med_DSPO', 'frequency_flag'],
      dtype='object')

In [49]:
#counting how many cust_ids
op_agged['customer_id'].nunique()

206209

In [50]:
#counting how many cust_ids in cust df 
cust['customer_id'].nunique()

206209

In [53]:
op_agged['customer_id'].dtype

dtype('int64')

In [54]:
cust['customer_id'].dtype

dtype('O')

In [55]:
#changing agged customer_id back to string to combine
op_agged['customer_id'] = op_agged['customer_id'].astype('str')

In [56]:
op_agged['customer_id'].dtype

dtype('O')

In [57]:
#combining op_agged and og cust. 
IC_merged = op_agged.merge(cust, on = ['customer_id'], indicator = True)

In [59]:
IC_merged.head()

,order_id,customer_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,...,First Name,Last Name,Gender,State,Age,date_joined,n_dependants,fam_status,income,_merge
0,2539329,1,1,2,8,NaN,196,1,0,Soda,...,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,both
1,2398795,1,2,3,7,15.0,196,1,1,Soda,...,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,both
2,473747,1,3,3,12,21.0,196,1,1,Soda,...,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,both
3,2254736,1,4,4,7,29.0,196,1,1,Soda,...,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,both
4,431534,1,5,4,15,28.0,196,1,1,Soda,...,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,both


In [61]:
#checking merge 
IC_merged['_merge'].value_counts(dropna = False)

both          32404859
left_only            0
right_only           0
Name: _merge, dtype: int64

In [62]:

# Export data to pkl
IC_merged.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'IC_merged_all.pkl'))


In [ ]:
#merging clean cut data 
IC_merged_sub = pd.merge(cust_clean, op_agged, on ='customer_id', how = 'left')